In [1]:
import sys
sys.path.insert(0, '/home/byu.local/danielkj/physics628/physics-628-exoplanet-stability/Spectral_Fraction_Code')
from IPython.display import display
from spectral_fraction import sf_AMD_simarch,sf_AMD_sim
from astropy import units as u
import rebound
import math
import numpy as np
from spock import FeatureClassifier, DeepRegressor
feature_model = FeatureClassifier()
deep_model = DeepRegressor()


def spockMetric(sim):
    l=sim.N
    stable=100
    planet=0
    for n in range(l-3):
    
        MetricSim=rebound.Simulation()
        MetricSim.add(sim.particles[0])
        MetricSim.add(sim.particles[n+1])
        MetricSim.add(sim.particles[n+2])
        MetricSim.add(sim.particles[n+3])
        model = FeatureClassifier()
        MetricSim.move_to_com()
        spM=model.predict_stable(MetricSim)
        
        MetricSim.move_to_hel()
        if spM < stable:
            stable=spM
            planet=n+1    
    return stable,planet

def spockMetricFull(sim):
    l=sim.N
    stable=100
    planet=0
    model = FeatureClassifier()
    sim.move_to_com()
    spM=model.predict_stable(sim)
    return spM

def hillStability(starData,planetData):
    mutualHill = []
    starMass = starData[0] 
    
    minSeparation = 0
    minpos = 0
    stable = False 
    
    for i in range(len(planetData)-1):
        mass1 = planetData[i][0]
        a1 = planetData[i][2]
        e1 = planetData[i][3]
        mass2 = planetData[i+1][0]
        a2 = planetData[i+1][2]
        e2 = planetData[i+1][3]
        
        mu1 = mass1/starMass
        mu2 = mass2/starMass
        
        rmh = math.pow((mu1+mu2)/3,1/3) * (a1+a2)/2
        
        nhill = (a2*(1-e2)-a1*(1+e1))/rmh
        
        mutualHill.append(nhill)
    
    minSeparation = min(mutualHill)
    aveSeparation = np.mean(mutualHill)
    minpos = mutualHill.index(minSeparation)
    
    if minSeparation >= 3.46:
        stable = True
    
    #print(minSeparation,minpos + 1,stable)
    
    return minSeparation,aveSeparation,minpos + 1,stable

def spectralFrac(sim):

    ##checks stability prediction from SPOCK
    model = FeatureClassifier()
    print('SPOCK prediction:',model.predict_stable(sim))

    p = sim.particles
    sim.integrator = 'WHFast'
    sim.dt = p[1].P/50 #50 timesteps in the inner orbit

    ##this method integrates the system independently and saves a Simulation Archive
    ##this allows for the SA to be used for other analysis and to be revisited without running the integration again
    t0 =  sim.t
    tmax = t0 + 5e6*p[1].P #run for 5 million orbits of inner planet
    Nsteps = 3000 #use same number of steps as VM20
    intvl = (tmax-t0)/Nsteps #interval to save snapshots in the simulation archive


    ##alternatively, this method integrates and outputs the spectral fraction directly from the simulation object
    print(sf_AMD_sim(sim))

def DeepRegressorMetric(sim):
    l=sim.N
    stabilityTimes = []
    for n in range(l-3):
        MetricSim=rebound.Simulation()
        MetricSim.add(sim.particles[0])
        MetricSim.add(sim.particles[n+1])
        MetricSim.add(sim.particles[n+2])
        MetricSim.add(sim.particles[n+3])
        dmodel = DeepRegressor()
        MetricSim.move_to_com()
        median,lower,upper = dmodel.predict_instability_time(MetricSim,10000)
        stabilityTimes.append(median)
    
    minTime = min(stabilityTimes)
    index = stabilityTimes.index(minTime)
    
    minTime = int(minTime)
    minPlanet = index + 1
    
    return minTime,minPlanet

def DeepRegressorMetricFull(sim):
    dmodel = DeepRegressor()
    sim.move_to_com()
    median,lower,upper = dmodel.predict_instability_time(sim,10000)
    return median,lower,upper



/home/byu.local/danielkj/.local/lib/python3.7/site-packages/pytorch_lightning/metrics/__init__.py:44: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  "`pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package"


In [9]:
starData = [1.02]

#planetData = [[1.95e-6,.01712218,.092358,.03478,.03323,2.11397,5.77121,4.1656]
#              ,[4.96e-6,.026809,.107976,.240898,.024457,3.7505,3.012178,2.920426]
#              ,[4.92e-5,.02978,.156314,.023048,.0190845,2.84487,1.697475,5.45357]
#              ,[1.59e-5,.035602,.19659,.007884,.018701,2.513814,.74625,5.745]
#              ,[8.44e-5,.02333802,.252889,.02974,.0196547,3.0725,2.4495,1.7554]
#              ,[.000209588,.03166,.47026,.0104,.0784,4.310367,2.53447,1.6898]]

#KOI 1613
#planetData = [[7.69e-7,.00763,.119743,.255715,.029521,1.448729,3.134967,5.203872]
#              ,[2.407e-7,.005303,.142536,.049023,.020538,4.8232,5.81334,5.1805]
#              ,[8.82e-7,.00586,.392318,.042841,.046326,1.54,3.610816,3.006721]]

#KOI 2045
#planetData = [[.0000119539,.015418,.053988,.092816,.050705,3.567351,1.220745,6.178164]
#              ,[7.2733623e-7,.011423,.11459,.169996,.030561,4.647298,.48522,.240408]
#              ,[6.35e-7,.012853,.145416,.064701,.180071,4.685787,.3064958,5.779418]]

#KOI 0041
#planetData = [[2.34e-6,.011487,.073127,.013632,.05433,3.164805,4.5784,1.27919]
#              ,[5.47e-5,.020517,.110633,.025425,.026053,6.079588,5.226191,.506975]
#              ,[1.48e-5,.013821,.217526,.056875,.028431,4.908769,4.344751,2.14516]]

#KOI 0474 constrained
planetData = [[5.71e-6,.011515,.058246,0,0,3.274296,.906788,4.216718],
             [5.83e-5,.02133,.097746,0,0,6.201815,2.785928,4.898061],
             [5.73e-6,.020121,.1871,0,0,5.506788,.873469,.499307],
             [.802624,1.048597,.707145,0,0,3.532923,3.408745,5.659677]]

#KOI 1574 constrained
#planetData = [[8.25e-6,.015072,.063841,0,0,1.14057,1.052588,3.821132],
#             [4.25e-6,.012851,.085091,0,0,2.270037,.298805,2.789106],
#             [5.761e-3,.093567,.465932,0,0,.279747,5.2147,3.46496],
#             [5.16e-4,.041947,.653939,0,0,1.595354,1.836623,2.878749]]



In [13]:
    sim = rebound.Simulation()
    sim.add(m=starData[0])
    
    for i in range(len(planetData)):
        
        mass = planetData[i][0]
        radius = planetData[i][1]
        semi = planetData[i][2]
        ecc = planetData[i][3]
        incline = planetData[i][4]
        o = planetData[i][5]
        O = planetData[i][6]
        meanAnom = planetData[i][7]

        sim.add(m = mass, r = radius, a = semi, e = ecc, inc = incline, Omega = O, omega = o, M = meanAnom)
    
    
    sim.move_to_com()
    #fig = rebound.OrbitPlot(sim)
    #display(fig)
    print(spockMetric(sim))
    print(spockMetricFull(sim))
    print(DeepRegressorMetric(sim))
    print(DeepRegressorMetricFull(sim))
    
    print(hillStability(starData,planetData))
        

(0.8189027, 2)
0.790519
(6535, 2)
(2325.9020389335674, 1166.0425042881316, 17760.220604895094)
(1.8169785277081227, 14.321508443425998, 3, False)


In [28]:
def hillStability(sim):
    mutualHill = []
    starMass = sim.particles[0].m
    nPlanets = sim.N - 1
    
    minSeparation = 0
    minpos = 0
    stable = False 
    
    for i in range(1,nPlanets):
        mass1 = sim.particles[i].m
        a1 = sim.particles[i].a
        e1 = sim.particles[i].e
        mass2 = sim.particles[i+1].m
        a2 = sim.particles[i+1].a
        e2 = sim.particles[i+1].e
        
        mu1 = mass1/starMass
        mu2 = mass2/starMass
        
        rmh = math.pow((mu1+mu2)/3,1/3) * (a1+a2)/2
        
        nhill = (a2*(1-e2)-a1*(1+e1))/rmh
        print(nhill)
        mutualHill.append(nhill)
    
    minSeparation = min(mutualHill)
    aveSeparation = np.mean(mutualHill)
    minpos = mutualHill.index(minSeparation)
    
    if minSeparation >= 3.46:
        stable = True
    
    #print(minSeparation,minpos + 1,stable)
    
    return minSeparation,aveSeparation,minpos + 1,stable 
    
    
    
sim2 = rebound.Simulation()
sim2.add(m=1.1)
sim2.add(m=4.776e-6, P=6.887/365 * 6.28, e=0.07, l=0.3)
sim2.add(m=1.5778e-5, P=12.815/365 * 6.28, e=0.017, l=2.8)
sim2.add(m=6.9927e-5, P=35.3335/365 * 6.28, e=0.0449, l=-0.5)
#sim2.move_to_com()
print(hillStability(sim2))
    
# XGBoost-based classifier

print(feature_model.predict_stable(sim2))
  
    


18.04421782928282
19.603542669653653
(18.04421782928282, 18.82388024946824, 1, True)
0.89363956


In [4]:
import rebound
from spock import FeatureClassifier, DeepRegressor
feature_model = FeatureClassifier()
deep_model = DeepRegressor()

sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03, l=0.3)
#sim.add(m=1.e-5, P=1.2, e=0.03, l=2.8)
sim.add(m=1.e-5, P=1.5, e=0.03, l=-0.5)
sim.move_to_com()

# XGBoost-based classifier
#print(feature_model.predict_stable(sim))


# Bayesian neural net-based regressor
median, lower, upper = deep_model.predict_instability_time(sim, samples=10000)
print(int(median))


AttributeError: SPOCK Error: SPOCK only works for systems with 3 or more planets

TypeError: close() argument must be a Figure, an int, a string, or None, not '%s'